In [ ]:
import pandas as pd
df = pd.read_csv('/content/train.csv')
df = df.dropna()
print(df.head())

   id  ... label
0   0  ...     1
1   1  ...     0
2   2  ...     1
3   3  ...     1
4   4  ...     1

[5 rows x 5 columns]


In [ ]:
# features
X = df.drop('label',axis=1) 
y = df['label']
X.shape,y.shape

((18285, 4), (18285,))

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
voc_size = 5000

One Hot 

In [ ]:
messages = X.copy()
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [ ]:
onehot_repr = [one_hot(words,voc_size)for words in corpus]
onehot_repr

[[2941, 2187, 2137, 4750, 2793, 2541, 1559, 2100, 1184, 3996],
 [4394, 1332, 4890, 123, 4043, 277, 191],
 [1192, 4633, 233, 4153],
 [1085, 2936, 1308, 4954, 4131, 2945],
 [3520, 4043, 932, 4622, 1846, 2229, 4043, 1768, 4227, 77],
 [4803,
  1212,
  4390,
  1167,
  215,
  62,
  3509,
  147,
  822,
  3991,
  2359,
  3767,
  4808,
  4657,
  191],
 [4540, 4876, 1295, 2276, 3380, 3645, 4446, 161, 996, 3486, 1771],
 [4882, 3429, 1468, 2786, 1124, 1114, 62, 462, 996, 3486, 1771],
 [3326, 3750, 3540, 726, 4117, 1957, 4915, 4198, 62, 599],
 [698, 1571, 4890, 1416, 429, 1130, 2866, 3063],
 [1196, 2906, 3120, 2282, 2474, 816, 1904, 4140, 807, 742, 2557],
 [4954, 1981, 2793, 1957, 62, 1124],
 [646, 1675, 2921, 3644, 1008, 3250, 1043, 2847, 4096],
 [2914, 2324, 4284, 3109, 1657, 262, 214, 996, 3486, 1771],
 [1574, 1114, 932, 3712, 682, 996, 3486, 1771],
 [3042, 1820, 728, 1736, 3566, 408, 3847, 2199, 3420, 3947],
 [995, 4357, 1332],
 [476, 3111, 3653, 3797, 62, 3024, 2716, 191],
 [2219, 3525, 4890, 

#Embedding Representation

In [ ]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 2100, 1184, 3996],
       [   0,    0,    0, ..., 4043,  277,  191],
       [   0,    0,    0, ..., 4633,  233, 4153],
       ...,
       [   0,    0,    0, ...,  996, 3486, 1771],
       [   0,    0,    0, ..., 3852,  575,  617],
       [   0,    0,    0, ..., 3302, 1209, 4009]], dtype=int32)

In [ ]:
len(embedded_docs)

18285

In [ ]:
#creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
# 1 LSTM with 100 neurons
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(18285, (18285,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

#Model Training

In [ ]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 9s 35ms/step - loss: 0.4875 - accuracy: 0.7402 - val_loss: 0.1963 - val_accuracy: 0.9127
Epoch 2/10
192/192 [==============================] - 6s 32ms/step - loss: 0.1488 - accuracy: 0.9426 - val_loss: 0.1888 - val_accuracy: 0.9205
Epoch 3/10
192/192 [==============================] - 6s 32ms/step - loss: 0.1075 - accuracy: 0.9625 - val_loss: 0.2155 - val_accuracy: 0.9191
Epoch 4/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0831 - accuracy: 0.9727 - val_loss: 0.2256 - val_accuracy: 0.9175
Epoch 5/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0676 - accuracy: 0.9777 - val_loss: 0.2546 - val_accuracy: 0.9163
Epoch 6/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0515 - accuracy: 0.9840 - val_loss: 0.2867 - val_accuracy: 0.9138
Epoch 7/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0431 - accuracy: 0.9864 - val_loss: 0.3247 - val_accuracy: 0.9135

##Adding Dropout

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

#Performance

In [ ]:
y_pred=model.predict_classes(X_test)

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,y_pred))

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred)*100)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[3120  299]
 [ 222 2394]]
91.36702568351284
